# Bronze layer - Dimension table

In [ ]:
in_parameter_run_id = 0
in_parameter_process_date = ""
in_parameter_table_name = ""
in_parameter_columns = ""
out_parameter_count_processed = 0

## 1. Load validation rules

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
df_validation_rules = spark.read.table("control.validation_rules").filter(col("table_name") == in_parameter_table_name).toPandas()

## 2. Extract data

In [ ]:
df = spark.read.parquet(f"Files/adventure_works_dw_2022/{in_parameter_table_name}/*.parquet")
df.select(in_parameter_columns.split(","))

In [ ]:
for index, row in df_validation_rules.iterrows():
    if row["column_type"] == "INT":
        df = df.withColumn(row["column_name_original"], col(row["column_name_original"]).cast(IntegerType()))
    elif row["column_type"] == "TINYINT" or row["column_type"] == "SMALLINT":
        df = df.withColumn(row["column_name_original"], col(row["column_name_original"]).cast(ShortType()))
    elif row["column_type"] == "DECIMAL":
        df = df.withColumn(row["column_name_original"], col(row["column_name_original"]).cast(DecimalType(row["column_size"], row["column_size_scale"])))
    elif row["column_type"] == "DATETIME":
        df = df.withColumn(row["column_name_original"], col(row["column_name_original"]).cast(TimestampType()))
    elif row["column_type"] == "DATE":
        df = df.withColumn(row["column_name_original"], col(row["column_name_original"]).cast(DateType()))
    else:
        df = df.withColumn(row["column_name_original"], col(row["column_name_original"]).cast(StringType()))

Add run identifier

In [ ]:
df = df.withColumn("process_date", lit(in_parameter_process_date))
df = df.withColumn("process_date", to_date("process_date", "yyyy-MM-dd"))
df = df.withColumn("run_id", lit(in_parameter_run_id))

## 2. Load data

In [ ]:
df.write.format("delta").mode("overwrite").saveAsTable(f"bronze.{in_parameter_table_name}")

In [ ]:
out_parameter_count_processed = df.count()

In [ ]:
mssparkutils.notebook.exit(out_parameter_count_processed)